# HOMEWORK 6 (10 points)


# Before we start: a few words about LLM-assisted coding

LLMs are great coding helpers, but as any assistants, they need supervision. After all, however much you rely on LLMs for coding, you are responsible for the result.

To help you a bit, we share some ideas that might improve your experience with LLMs.

**Vibe coding guidelines**

We'd recommend trying **Anthropic Claude 3.7 Sonnet**, or **ChatGPT o3/o4-mini**, or **Gemini 2.5** - they'll give you the best result. **DeepSeek V3** or **R1** should also work well. A **playground** is a better vibe coding interface than an API, especially because you'll likely need several iterations to polish the code. Unless you use an AI-powered IDE such as **Cursor**, of course.

Here are some general prompting guidelines for LLM-assisted coding:

1. **Clearly explain which functionality and interface you need**

  "I need a chatbot" is too vague, and the results will be unpredictable. Describe how the user will be interacting with the chatbot. Explain which parameters to set up in the constructor. Choose whether you want a function or a class and clearly communicate this. Decide how exceptions should be treated.

  Some of the LLMs will be all too earger to create many things you don't ask them - a productionalizing framework, a chatbot factory, examples of usage etc. Without proper guidance, they can swamp you in code. To avoid this, you may add very insistently that you only want the chatbot class/function and nothing else.

  Since we're working in Jupyter, LLMs may annoy you much by creating usage examples that require command line execution. Explaining how you are going to work with the code might help with that.

2. **Provide code examples**

  If you're ok with the design of `answer_with_llm` and if you want the new class or function to have a similar interface, provide its implementation. LLMs are usually good at reproducing design patterns.

  It's a good practice to highlight code with

  ````{verbatim}
  ```
  <your code>
  ```
  ````

3. **Test LLM's understanding**

  I personally like requesting an LLM to ask any questions it had BEFORE (yes, caps won't hurt) it starts generating code. This might help you to steer the LLM into the right direction. From our experience LLMs sometimes ask really good questions here, uncovering things we'd forgotten to think of beforehand.

4. **Be ready for several iterations of improvement**

  Even if you prompt an LLM really carefully, it may still surprise you. So, though in this task you may grab the first working version, we advise you not to rely blindly on whatever LLMs generate, especially in longer projects, where programming antipatterns might cost you dearly.

  From our experience LLMs are reasonably good at writing boilerplate code, but look out for code duplication, hardcoding, and overcomplication.

# Setting things up

Before starting to work, save your Nebius AI Studio API key to a file called `nebius_api_key` (use plain text editor for that to avoid adding rogue characters) and load it to colab.

In [ ]:
!pip install -q openai

In [ ]:
import os

with open("nebius_api_key", "r") as file:
    nebius_api_key = file.read().strip()

os.environ["NEBIUS_API_KEY"] = nebius_api_key

We'll be calling APIs quite often in this notebook, so let's define a shortcut fuction to avoid repeating all the code. Also, we'll prettify the output in such a way that it can be viewed without scrolling right.

In [ ]:
from openai import OpenAI

# Nebius uses the same OpenAI() class, but with additional details
nebius_client = OpenAI(
    base_url="https://api.studio.nebius.ai/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
)

llama_8b_model = "meta-llama/Meta-Llama-3.1-8B-Instruct"

def prettify_string(text, max_line_length=80):
    """Prints a string with line breaks at spaces to prevent horizontal scrolling.

    Args:
        text: The string to print.
        max_line_length: The maximum length of each line.
    """

    output_lines = []
    lines = text.split("\n")
    for line in lines:
        current_line = ""
        words = line.split()
        for word in words:
            if len(current_line) + len(word) + 1 <= max_line_length:
                current_line += word + " "
            else:
                output_lines.append(current_line.strip())
                current_line = word + " "
        output_lines.append(current_line.strip())  # Append the last line
    return "\n".join(output_lines)

def answer_with_llm(prompt: str,
                    system_prompt="You are a helpful assistant",
                    max_tokens=512,
                    client=nebius_client,
                    model=llama_8b_model,
                    prettify=True,
                    temperature=0.7) -> str:

    messages = []

    if system_prompt:
        messages.append(
            {
                "role": "system",
                "content": system_prompt
            }
        )

    messages.append(
        {
            "role": "user",
            "content": prompt
        }
    )

    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=max_tokens,
        temperature=temperature
    )

    if prettify:
        return prettify_string(completion.choices[0].message.content)
    else:
        return completion.choices[0].message.content

## Task 1. Benchmarking LLMs on GSM8k (5 points)

In this task, you'll benchmark several LLMs against the [GSM8k](https://huggingface.co/datasets/openai/gsm8k) dataset. It contains grade school math problems which should be reatively easy for the LLMs (or not?).

To start with, let's download the dataset and check several problems from it.

In [ ]:
!pip install -q -U datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset("openai/gsm8k", "main", cache_dir="./gsm8k")

In [ ]:
# Basic info
print("Dataset structure:", dataset)
print("Available splits:", list(dataset.keys()))
print("Train examples:", len(dataset['train']))
print("Test examples:", len(dataset['test']))

# Look at the first example
example = dataset['train'][0]
print("\n--- Example Problem ---")
print("Question:", example['question'])
print("\nAnswer:", example['answer'])

Your task will be to create an `GSM8KEvaluator` class with the following interface:

* `__init__` loads the test split of the dataset
* `run_evaluation(self, client, model, n, n_questions)` tests a model `model` offered by a client `client` on the dataset, calculating:
  
  * Accuracy - (number of problems on which the LLM arrives at a correct answer) / 30
  * Average execution time (seconds per problem)
  * Average solution length in tokens

  The `n` parameter is the number of **Self consistency** passes: for each problem, you'll need to get `n` solutions and choose the most popular answer for evaluation. We suggest using the parameter `n` in

  ```
  client.chat.completions.create(
                messages,
                model=model,
                n=n
            )
  ```

  The `n_questions` parameter determines how many first questions to score with a model. Don't take `n_questions` more than 50 to save time and money.

You can draw inspiration from the `MMLUEvaluator` class implemented [here](https://colab.research.google.com/github/Nebius-Academy/LLM-Engineering-Essentials/blob/main/topic1/1.5_how_to_choose_an_llm.ipynb).

Test at least three LLMs **on the first 50 problems from the test split** - we'd suggest trying one small (4-8B parameters), one medium (30-70B parameters) and one huge (>200B parameters) model. Run them with `n = 1, 5`. Compare the accuracy.

**Hints and suggestions**

0. Note that you'll need to extract the correct answer from the solution. Luckily, it should be quite easy to do.
1. Be careful with parsing the LLM's answer - wrong parsing can greatly worsen the results
2. Be careful with the `max_tokens` parameter - if you set it too low, the solutions might get cut short, especially with larger models and especially with long reasoning ones.
3. I'd avoid **DeepSeek R1** for this task. Luckily, **Qwen 3** models are also reasoning, and they are much smaller.
4. Use `tqdm` to create evaluation progress bars. Watching a progress bar moving is reassuring, especially for larger models that work for longer time.
5. Log all the model answers and return them together with metrics. Start with `n_questions=3` or something like that to debug your evaluator (and especially your parsing).

In [ ]:
# <YOUR CODE HERE>

## Task 2. Mapping LLM "thoughts" (5 points)

In this task, we'll look closer at "thinking patters" of LLMs:

- We'll look closer at an LLM's "tree of thoughts",
- We'll investigate how long the typical thoughts are,
- We'll explore the "underthinking" phenomenon.

If you encounter any difficulties or simply want to see our solutions, feel free to check the [Solutions notebook](https://colab.research.google.com/github/Nebius-Academy/LLM-Engineering-Essentials/blob/main/topic2/r.1_intro_to_llm_reasoning.ipynb).

To have something to experiment with, we'll run evaluation of **QwQ-32B-Preview** on a subset of [MATH benchmark](https://huggingface.co/datasets/nlile/hendrycks-MATH-benchmark). This benchmark is relatively challenging, but to a reasonable extent. It's not [FrontierMath](https://epoch.ai/frontiermath) :)

We'll take the first 50 problems that satisfy two following conditions:

- Their answer is either straightforwardly converted to `float`, or it's a simple Latex-formatted fraction, like `\frac{2}{3}`.
- Their "level" is either 4 ot 5 (more challenge!).

**Note** You can use **deepseek-ai/DeepSeek-R1**, if you want, but it will generate solutions *very* slowly (not mentioning the cost).

Also, if you don't want to run the evaluation of **QWQ** on your own, you may download the `qwq_results.pkl` file from Google drive:

In [ ]:
!gdown 1_hEX_h7fj6FXH3lG5AMthJjiK1JwAU9J

### Evaluating QWQ on MATH Dataset

If you're in, let's create the evaluator. And we'll start by data preprocessing.

In [ ]:
!pip install -q datasets

In [ ]:
from datasets import load_dataset
ds = load_dataset('nlile/hendrycks-MATH-benchmark', split='test')

In [ ]:
import re

def conver_string_to_number(s):
    """
    Checks if a string is a number or a fraction and computes the fraction if applicable.

    Args:
        s: The input string.

    Returns:
        A float representing the number or fraction, or None if the string is invalid.
    """
    if "_" in s:
        return None

    try:
        return float(s)  # Try converting to a float directly
    except ValueError:
        match = re.match(r"\\frac\{(\d+)\}\{(\d+)\}", s)
        if match:
            numerator = int(match.group(1))
            denominator = int(match.group(2))
            if denominator != 0:
                return numerator / denominator
            else:
                return None  # Handle division by zero
        else:
            return None  # String is not a number or a valid fraction

# Example usage
strings = ["123", "\\frac{1}{2}", "\\frac{3}{0}", "abc", "\\frac{4}{5}"]
for s in strings:
    result = conver_string_to_number(s)
    if result is not None:
        print(f"'{s}' is a valid number or fraction. Result: {result}")
    else:
        print(f"'{s}' is not a valid number or fraction.")


In [ ]:
import pandas as pd


df = pd.DataFrame(ds)

df['num_answer'] = df['answer'].apply(conver_string_to_number)
df['valid_answer'] = df['num_answer'].notna()

# Select the first 50 rows where the 'answer' column passes the check
selected_rows = df[(df['valid_answer']) & (df['level'] >= 4)].head(50)
selected_rows

The evaluator itself:

In [ ]:
import pandas as pd
from typing import List, Dict, Tuple
import json
from pathlib import Path
import numpy as np
from tqdm import tqdm
import re

from openai import OpenAI

from datasets import load_dataset

def find_boxed_content(text):
    matches = re.findall(r"boxed\{(.*?)\}", text)
    try:
        return conver_string_to_number(matches[-1])
    except:
        try:
            return conver_string_to_number(text.split("\n")[-1].split(" ")[-1].strip(".;$"))
        except:
            print(f"""Wrong format in:
                {text.split()[-1]}""")
            return None

class MATHEvaluator:
    def __init__(self, system_prompt: str = "You are a helpful assistant.",
                 prompt: str = None):
        """
        Initialize the MATH evaluator.

        Args:
            system_prompt: Optional system prompt for the model
            prompt: Custom prompt for the model
        """

        self.system_prompt = system_prompt

        self.prompt = """{question}"""

        self.questions, self.answers = selected_rows["problem"].to_list(), selected_rows["num_answer"].to_list()

    def extract_answer(self, solution: str) -> str:
        """
        Extract the letter answer from model's response.

        Args:
            response: Raw model response

        Returns:
            Extracted answer (an float)
        """
        # Look for a single letter answer in the response
        try:
            # answer = float(solution.split("\n").split(" ")[1].strip(".;)"))
            answer = find_boxed_content(solution)
        except:
            answer = None
        # print(solution.split("\n")[-1])
        # print(answer)
        return answer

    def evaluate_single_question(self, question: str,
                                 correct_answer: float,
                                 client, model, max_tokens,
                                 temperature=None) -> Tuple[bool, str]:
        """
        Evaluate a single question.

        Args:
            question: Formatted question string
            correct_answer: Correct answer letter

        Returns:
            Tuple of (is_correct, extracted_answer, model_response)
        """
        try:
            model_response = answer_with_llm(
                prompt=self.prompt.format(
                    question=question
                ),
                client=client, model=model,
                system_prompt=self.system_prompt,
                max_tokens=max_tokens
            )
            answer = self.extract_answer(model_response)
            if answer:
                is_correct = np.abs(answer - correct_answer) < 1e-10
            else:
                is_correct = False
            return is_correct, answer, model_response
        except Exception as e:
            print(f"Error evaluating question: {e}")
            return False, None, None

    def run_evaluation(self, client : OpenAI, model : str,
                       n_questions=50, max_tokens=8192, temperature=0.) -> Dict:
        """
        Run evaluation of a given model on the first n_questions.

        Args:
            client: Which client to use (OpenAI or Nebius)
            model: Which model to use
            n_questions: How many first questions to take

        Returns:
            Dictionary with evaluation metrics
        """
        evaluation_log = []
        correct_count = 0
        correct_format_count = 0
        if n_questions:
            n_questions = min(n_questions, len(self.questions))
        else:
            n_questions = len(self.questions)

        for i in tqdm(range(n_questions)):
            is_correct, answer, model_response = self.evaluate_single_question(
                question=self.questions[i],
                correct_answer=self.answers[i],
                client=client,
                model=model,
                max_tokens=max_tokens,
                temperature=temperature
            )

            if answer:
                correct_format_count += 1

            if is_correct:
                correct_count += 1

            evaluation_log.append({
                'answer': answer,
                'correct_format': not answer is None,
                'model_response': model_response,
                'is_correct': is_correct
            })

        accuracy = correct_count / n_questions
        format_correctness = correct_format_count / n_questions
        evaluation_results = {
            'accuracy': accuracy,
            'format_correctness': format_correctness,
            'evaluation_log': evaluation_log
        }

        return evaluation_results

In [ ]:
math_evaluator = MATHEvaluator(system_prompt=None)

In [ ]:
client = OpenAI(
    base_url="https://api.studio.nebius.ai/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
)

qwq_results = math_evaluator.run_evaluation(
    client=client,
    model="Qwen/QwQ-32B-Preview",
    n_questions=None,
    max_tokens=None
)
print(f'\nAccuracy: {deepseek_results["accuracy"]}')

Let's save the results to file:

In [ ]:
import pickle

pickle.dump(qwq_results, open("qwq_results.pkl", "wb"))

Now, you can load the file even if you didn't create it:

In [ ]:
!gdown 1_hEX_h7fj6FXH3lG5AMthJjiK1JwAU9J

In [ ]:
import pickle
qwq_results = pickle.load(open("qwq_results.pkl", "rb"))

### Analyzing thoughts

We've prepared quite a large thought analysis and visualization script; so we decided not to include it here (please check it in github if you're curious). Here, we'll only download and import it from `thought_analysis.py`.

A few words about what's happening in `thought_analysis.py`:

- First of all, if the solution has `<think>...</think>` markup inside, only the fragment between them is extracted. (We're only interested in the "internal" thinking process.)
- Then, solutions are divided into individial "thoughts" using the following heuristics:
  - `Alternatively`, `Wait`, `But wait`, `But let me check again`, `But let's verify`, and similar phrases mark the starts of new "thoughts". Note that they are typical indications of backtracking and solution branching. There may be more, of course.
  - Otherwise, a "thought" is a continuous range of paragraphs of length not less than `min_split_size` characters (we'll take `min_split_size=120`). Separate-line Latex formulas are always added to the previous thought.
- For each "thought", its length in tokens is calculated. For that, we need to supply the right **tokenizer** which corresponds to the model which generated the solutions - in our case, **[QwQ-32B-Preview](https://huggingface.co/Qwen/QwQ-32B-Preview)**. And that's why you needed to create a **Hugging Face access token**. If you haven't done it yet, please register to HF, get the token and load it to colab in a `hf_access_token` file.

  If you're ardently against registering to Hugging Face, you can supply `None` as `tokenizer`, but in this case you won't get correct token length of individual "thoughts".

- The **"thee of thoughts"** is constructed in the following way:
  - If a "thought" starts with `Alternatively`, `Wait`, or `But wait`, we query using another LLM (**Llama-3.1-70B** by default) to determine which of the previous "thoughts" is continued by this. If it starts a completely different solution, the thought is connected to **root** (empty solution; the very start).
  - Otherwise, the thought is connected with the previous one.

- Finally, the tree is saved as `thought_analysis/thought_tree.png`, if you didn't change default output path.

In [ ]:
!pip install -q openai

In [ ]:
!curl -o thought_analysis.py https://raw.githubusercontent.com/Nebius-Academy/LLM-Engineering-Essentials/main/topic2/thought_analysis.py

Let's also download a sample solution for us to work with.

In [ ]:
!curl -o sample_solution.txt https://raw.githubusercontent.com/Nebius-Academy/LLM-Engineering-Essentials/main/topic2/sample_solution.txt

In [ ]:
from openai import OpenAI
from thought_analysis import analyze_solution_thoughts
from transformers import AutoTokenizer

with open("hf_access_token", "r") as file:
    hf_access_token = file.read().strip()

client = OpenAI(
    base_url="https://api.studio.nebius.ai/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY")
)
model = "meta-llama/Meta-Llama-3.1-70B-Instruct"

reasoning_model = "deepseek-ai/DeepSeek-R1"
tokenizer = AutoTokenizer.from_pretrained(reasoning_model,
                                          hf_access_token=hf_access_token)

file_path = "sample_solution.txt"  # Path to the solution file
output_dir = "thought_analysis"  # Directory to save results

with open(file_path, 'r', encoding='utf-8') as file:
    solution_text = file.read()


connections, viz_path, summary_path = analyze_solution_thoughts(
    solution_text,
    client=client,
    model=model,
    tokenizer=tokenizer,
    output_dir=output_dir,
    min_split_size=120
)

We can look closely at the connection or just check the `.png` file.

In [ ]:
connections

**Your task**. Take some other solutions, construct their trees of thoughts.

You may also play with solutions generated by **DeepSeek R1**. We created several of them for you:

- 10 from the MATH benchmark,
- 10 from the AIME benchmark,
- 2 from the Frontier Math benchmark.

Just beware that these solutions will be much, much longer than the solutions by **QWQ**.

You can download them, like this:

In [ ]:
!gdown 1TpROB-8XAE6z1OlTfli7XB6YoY6WQi18
!unzip deepseek_solutions.zip -d deepseek_solutions/

And, of course, feel free to create your own solutions!

### Size of Branches of Thoughts

**This is a task for you**! Create a simplified fuction

```
thoughts, token_counts = decompose_solution_thoughts(sample_text,
                                                     tokenizer=tokenizer)
```

that, given a solution `sample_text` and a tokenizer, returns:

- `thoughts` which is a split of `sample_text` by exactly `Alternatively`, `Wait`, `But wait` (you may add some of their synonyms you'll spot in the solutions). So, we only keep track of the **whole branches of the "tree of thoughts"** here.
- `token_counts` - the number of tokens in each of these "branches".

Now, we suggest you to explore the size of these branches in tokens. Create a histogram of branch sizes. What can you say about its shape? Take a look at several extremely long branches - what do you think, why are they so long? Check the shortest branches. What happens there?

In [ ]:
# <YOUR EXPERIMENTS HERE>

### LLM underthinking

This part is inspired by [Thoughts Are All Over the Place: On the Underthinking of o1-Like LLMs](https://arxiv.org/pdf/2501.18585). This paper investigated connection between the length of thought branches (fragments of solution between "Alternatively", "Wait", etc) and the solution accuracy. The found out that in many cases LLMs abandon promising solutions, cutting thought branches short before they could come to fruition - and this might contribute to failure of the whole solution.

**Your task**: create on one plot two histograms of branch lengths - one histogram for tasks with correct answer and one of tasks with incorrect answer. You can find information about answer correctness in `qwq_results["evaluation_log"]` (`"is_correct"` fields).

Since there is a different number of correct and incorrect answers in the data, we recommend normalizing the histograms so that they show frequency instead of count. This may be done by setting `density=True`. Do you see any specific patterns?

In [ ]:
# <Your experiments here>